In [1]:
import utils
import pandas as pd
import numpy as np

## Load and Clean Data

In [2]:
data = pd.read_csv('../data/namdinh.csv').fillna(0)

/Users/msanch35/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (27,33,61,65,69,71,81,82,103,104,106,107,113,114,116,117,120,132,133,134) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
drop = ['border', 
         'covid', 
         'fibrosis', 
         'pulmonary_scar', 
         'lung_opacity', 
         'airspace_opacity', 
         'other_findings']

findings = []
for i in list(data.columns):
    if 'ai_' == i[0:3]:
        add = True
        for j in drop:
            if j in i:
                add=False
        if add:    
            findings.append(i[3::])

### Merge Eqiuvalent Findings


In [4]:
data['opacity'] = data[['lung_opacity', 'airspace_opacity']].max(axis=1)
data['ai_opacity'] = data[['ai_lung_opacity', 'ai_airspace_opacity']].max(axis=1)

data['lung_scar'] = data[['fibrosis', 'pulmonary_scar']].max(axis=1)
data['ai_lung_scar'] = data[['ai_fibrosis', 'ai_pulmonary_scar']].max(axis=1)

### Set Clinically Significant Findings

In [5]:
sig_findings = [
    'opacity',
    'cardiomegaly',
    'lung_lesion',
    'pleural_effusion',
    'pneumothorax',
    'consolidation',
    'pneumonia',
    'edema',
    'cavitation',
    'tuberculosis',
    'nodule',
    'mass',
    'mass_nodule',
    'pleural_other'
]
non_sig_findings = [i for i in findings if i not in sig_findings]
non_sig_findings

['no_finding',
 'fracture',
 'atelectasis',
 'enlarged_cardiomediastinum',
 'widening_mediastinum',
 'medical_device']

## Compute Average Prevalence Over All Findings

In [6]:
avg_p_find = 0
for i in range(len(findings)):
    p_find = len(data[data[findings[i]] ==1])
    avg_p_find += p_find
avg_p_find /= len(findings) * len(data)
print(avg_p_find)

0.05544516983631375


## Compute Average Prevalence Of Disagreement

In [7]:
avg_p_dis = 0
for i in range(len(findings)):
    p_dis = len(data[data[findings[i]] != data['ai_'+findings[i]]]) 
    avg_p_dis += p_dis
avg_p_dis /= len(findings) * len(data)
print(avg_p_dis)

0.05353790380009063


## Compute Average True Positive and False Positive Rates of the Diagnostic Model

In [8]:
avg_TPR_f = 0
den = 0
for i in range(len(findings)):
    if len(data[data[findings[i]]==1]) > 0:
        avg_TPR_f += len(data[(data[findings[i]] == data['ai_'+findings[i]]) & (data[findings[i]]==1)])
        den +=  len(data[data[findings[i]]==1])
avg_TPR_f /= den
print(avg_TPR_f)

0.729387461828633


In [9]:
avg_FPR_f = 0
den = 0
for i in range(len(findings)):
    if len(data[data[findings[i]]==1]) > 0:
        avg_FPR_f += len(data[(data[findings[i]] != data['ai_'+findings[i]]) & (data[findings[i]]==0)])
        den += len(data[data[findings[i]]==0])
avg_FPR_f /= den
print(avg_FPR_f)

0.04079566842579754
